<a href="https://colab.research.google.com/github/marriamaslova/ML_homeworks/blob/main/ml_hw_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# импорты необходимых библиотек
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

# import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [4]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize

In [ ]:
import nltk
nltk.download('punkt')

Подготовка данных

In [6]:
data = pd.read_csv('IMDB-Movie-Data.csv')

In [7]:
meta_mean = data.Metascore.mean()
rev_mean = data['Revenue (Millions)'].mean()

In [8]:
data.Metascore.fillna(meta_mean, inplace=True)
data['Revenue (Millions)'].fillna(rev_mean, inplace=True)

In [9]:
data["text"] = data.Description.apply(lambda x: word_tokenize(x))

In [10]:
for text in data["text"]:
  for elem in text:
    if elem.isalnum():
      continue
    else:
      text.remove(elem)

In [11]:
input_text = list(data.text.values)
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(input_text)]

In [ ]:
model = Doc2Vec(documents, vector_size=5, window=3, min_count=1, workers=4)

In [13]:
model.save("D2V.model")

In [14]:
vectors = []
for x in documents:
    vec = list(model[x.tags][0])
    vectors.append(vec)

In [15]:
split_df = pd.DataFrame(vectors,
                        columns=['v1', 'v2', 'v3','v4',"v5"])

In [16]:
result = data.join(split_df, how='left')

In [113]:
data_sm = result[[
                'Rating', 'Votes',
                'Revenue (Millions)', "v1","v2","v3","v4","v5"]
              ] #'Runtime (Minutes)',"Year", 'Metascore',

Подготовка матриц

In [114]:
X = data_sm.drop(["Rating"],axis=1).values 

In [115]:
y = data_sm['Rating'].values

In [116]:
sc = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(sc.fit_transform(X), y, random_state=42)

Обучение моделей

Linear Regression

In [117]:
regressor_1 = LinearRegression() 
regressor_1.fit(X_train, y_train)
y_preds_1 = regressor_1.predict(X_test)

In [118]:
mean_absolute_error(y_test, y_preds_1) 

0.6504364920232626

In [119]:
mean_squared_error(y_test, y_preds_1) 

0.7384124087498499

In [120]:
mean_squared_error(y_test, y_preds_1, squared=False) 

0.8593092625765475

С данными, прошедшими nltk_токенизацию и удаление пунктуации:

0.48844185138116986

0.4680083001667433

0.6841113214724218

С удалением колонок 'runtime' и 'year':

0.5032091385237876

0.4829745484817905

0.6949637029959123

С удалением 'v4', 'v5':

0.5035543297977305

0.4833776216079181

0.6952536383277099

С удалением 'Metascore':

0.6492394802672429

0.7359858349056743

0.8578961679047612

С возвращением 'v4', 'v5'

0.6504364920232626

0.7384124087498499

0.8593092625765475

Ridge

In [128]:
regressor_2 = Ridge(alpha=50) 
regressor_2.fit(X_train, y_train)
y_preds_2 = regressor_2.predict(X_test)

In [129]:
mean_absolute_error(y_test, y_preds_2) 

0.6527083420329944

In [130]:
mean_squared_error(y_test, y_preds_2) 

0.737961673041615

In [131]:
mean_squared_error(y_test, y_preds_2, squared=False) 

0.8590469562495493

С дефолтным Alpha:

0.4884393707190119

0.4680851794091884

0.6841675082968998

С Alpha = 50:

0.48938148827901207

0.47286345615287706

0.6876506788718215

С удалением колонок 'runtime' и 'year':

0.5035001130914656

0.4859694234632252

0.6971150718950389

С удалением 'v4', 'v5':

0.5036599748546738

0.4862129011709035

0.6972896823924067

С удалением 'Metascore':

0.6515758220205519

0.7352336456784185

0.8574576640735206

С возвращением 'v4', 'v5':

0.6527083420329944

0.737961673041615

0.8590469562495493

Lasso

In [132]:
regressor_3 = Lasso() 
regressor_3.fit(X_train, y_train)
y_preds_3 = regressor_3.predict(X_test)

In [133]:
mean_absolute_error(y_test, y_preds_3) 

0.7736000000000001

In [134]:
mean_squared_error(y_test, y_preds_3) 

0.9734313244444442

In [135]:
mean_squared_error(y_test, y_preds_3, squared=False) 

0.9866262334057636

С дефолтным Alpha:

0.7736000000000001

0.9734313244444442

0.9866262334057636

При изменении Alpha, как ни странно, результаты оценки не меняются

С удалением колонок тоже ничего не меняется

-----------------------------------------------------------------

Я думаю, что провела достаточно экспериментов, чтобы делать выводы. 

Во-первых, всегда лучше работать с данными, которые хорошо почищены. Поэтому важно было удалить пунктуацию и токенизировать при помощи библиотеки для работы с текстами. 

Во-вторых, настройки alpha в данном случае не повлияли на результаты оценки критически. Сразу можно отметить модель Lasso, которая, видимо, уже при дефолтных настройках  достигла таких оценок, которые было не "перебить". И эти значения остались самыми высокими для всех попробованных моделей.

Изменение состава датасета для модели сильно повлияло на оценки работы Linear Regression и Ridge. Общий вывод - чем меньше колонок со всякими оценками мы включаем, тем лучше работает модель. Оставив только Votes, Revenue и векторы, я получила лучшие оценки для двух первых моделей. Причем удаление части векторов или возвращение их назад на оценку влияют слабо. В целом можно сказать, что при наличии совсем небольшого количества параметров, модель лучше предсказывает зависимость. 

При такой работе с параметрами получается, что оценка обычной LinearRegression находится очень рядом с оценкой Ridge. 